In [344]:
import sqlite3
import csv
from contextlib import closing

FILE_NAME = "productlist.csv"
#BOOK, PROD001, The Rainbow Fish, J. Alison James, 12.99, 30

class Product:
    category = "Product"
    def __init__(self, category, pid, name, price, qty):
        self.pid = pid
        self.name = name
        self.price = price
        self.qty = qty
        
class Book(Product):
    category = "BOOK"    
    def __initi__(self, category, pid, name, author, price, qty):
        super().__init__(pid, name, price, qty)
        self.author = author

        
    
class Movie(Product):
    category = "MOVIE"
    def ___init__(self, category, pid, name, price, genre, year, qty):
        super().__init__(pid, name, price, qty)
        self.__genre = genre
        self.__year = year
        
    def get_genre(self):
        return self.__genre
    
    def get_year(self):
        return self.__year


def save_to_file_menu(dbconn, filename) :
    """retrieve data from database and save data to a csv file"""
    
    with dbconn :
        with closing(dbconn.cursor()) as db :
            db.execute("SELECT * FROM Products")

            with  open(filename, "w") as out_csv_file : 
                csv_out = csv.writer(out_csv_file)
                for item in db.fetchall() :
                    csv_out.writerow(item)

def initalize_from_file(dbconn, filename, products) :
    """reads csv file and save data into database and list of all employees"""
    
    with  open(filename, "r") as data_file : 
        csv_data = csv.reader(data_file)
            
        with closing(dbconn.cursor()) as db :            
            for item in csv_data :
                if (len(item) == 6) :
                    db.execute("INSERT INTO Products VALUES(?, ?, ?, ?,'','', ?, ?)", 
                               (item[0], item[1], item[2], item[3],item[4], item[5]))
                    dbconn.commit()
                    product = Book(item[1], item[2], item[3], item[4],item[5])
                    products.append(product)
                elif (len(item) == 7) :
                    db.execute("INSERT INTO Products VALUES(?, ?, ?, '', ?, ?, ?, ?)", 
                               (item[0], item[1], item[2], item[3], item[4], item[5], item[6]))
                    dbconn.commit()
                    product = Movie(item[1], item[2], item[4], item[5], item[6])
                    products.append(product)
                else :
                    print(f"Invalid file format. Skipping...{item}...")

def get_all_products(dbconn) :
    """retrieves all products data from database"""

    # check if db connection exists
    with dbconn:
        db.execute("SELECT * FROM Products")
        return db.fetchall()
    
def list_all_products_menu(dbconn) :
    """lists all products' information"""
    
    retval = get_all_products(dbconn)
    print("*** All Products ***")
    print(f' PRODUCT  NAME                                   PRICE          QTY       TOTAL')
    print('='*80)
    cost=0
    for item in retval :
        print(f"{item[1]} {item[2].ljust(38)}\t${item[6]}\t\t{item[7]}\t  ${round(item[6]*item[7],2)}")
        cost += round(item[6]*item[7],2)
    print('-'*80)
    print(f'TOTAL:\t\t\t\t\t\t\t\t\t  ${cost}')


#category, pid, name, price, genre, year, qty
def add_product_menu(dbconn, allproducts):
    """adds a new product to the table"""
    with dbconn:
        select = input('Would you like to add a movie or a book? "type book or movie": ')
        select = select.upper()
        if select == 'MOVIE':
            name = input("Enter the name of the movie: ")
            pid = input("Enter the Poduct ID: it should be 'PROD###': ")
            price = float(input("Enter the movie's price: dont enter $ or , "))
            genre = input("Enter the genre: ")
            year = input("Enter the year of release: ")
            qty = input("Enter the quantity")
            db.execute("INSERT INTO Products VALUES (?, ?, ?, NULL, ?, ?, ?, ?)", 
                               (select, pid, name, genre, year, price, qty))
            dbconn.commit()
            
        elif select == 'BOOK':
            name = input("Enter the name of the book: ")
            pid = input("Enter the Poduct ID: it should be 'PROD###': ")
            author = input("Enter the name of the Author: ")
            price = float(input("Enter the movie's price: dont enter $ or , "))
            qty = input("Enter the quantity")
            db.execute("INSERT INTO Products VALUES (?, ?, ?, ?,NULL,NULL, ?, ?)", 
                               (select, pid, name, author, price, qty))
            dbconn.commit()
        else:
            print('Please enter either movie or book')
            
       
       
                   



In [345]:
#In memory database
dbconn = sqlite3.connect(':memory:')

# set a variable up for my cursor so I can write SQL scripts
db = dbconn.cursor()

# create Products table
db.execute("""CREATE TABLE Products (
            category TEXT,
            pid TEXT,
            name TEXT,
            author TEXT,
            genre TEXT,
            year INTEGER,
            price REAL,
            qty INTEGER
            )""")

#commits your transactions into the DATABASE
dbconn.commit()

In [346]:
allproducts = []
initalize_from_file(dbconn, FILE_NAME, allproducts)



In [347]:
def menu():
    ans = 0
    while ans != 5:
        ans = int(input('''1. Show all products\n2. Add a product\n3. Update a product\n4. Delete a product\n5. Exit
                       '''))
        if ans == 1:
            list_all_products_menu(dbconn)
            
        elif ans == 2:
            add_product_menu(dbconn, allproducts)
        
        elif ans == 3:
            print (f'This fucntion is not available at this time. Please select another option.\n')
            continue
        elif ans == 4:
            print (f'This fucntion is not available at this time. Please select another option.\n')
            continue
        elif ans == 5:
            print('Thank you for using this program. Have a nice day.')
            
        else:
            print('Please chose one of the 5 options listed above.')
            continue
menu()

1. Show all products
2. Add a product
3. Update a product
4. Delete a product
5. Exit
                       5
Thank you for using this program. Have a nice day.


In [349]:
FILE_NAME2 = "productlist2.csv"
save_to_file_menu(dbconn, FILE_NAME2)  
# must close your connection before exiting your program. 
dbconn.close()